
Search the best model ⚙️
----------------------------

This notebook will use the simple Bayesian Optimization algorithm implemented in [bayesian_optimization](bayesian_optimization.ipynb). To achieve this, we must define the primary hyperparameters and their corresponding search spaces. We must also define functions to help us search for the best combination of hyperparameters and display the results as a parallel coordinate chart in the tensorboard.


Let us begin with the definition of the hyperparameters to tune.

In [1]:
# import some libraries
import warnings
warnings.filterwarnings('ignore')

from fake_face_detection.optimization.fake_face_bayesian_optimization import SimpleBayesianOptimizationForFakeReal
from fake_face_detection.data.fake_face_dataset import FakeFaceDetectionDataset
from fake_face_detection.trainers.custom_trainer import get_custom_trainer
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer
from torchvision.transforms import Compose, transforms
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from functools import partial
import pickle
import torch
import os

# pl.seed_everything(0)

# disable wandb
os.environ["WANDB_DISABLED"] = "true"

### Hyperparameters

The following hyperparameters will be necessary to train the model:

- The batch size $ \in \{8, 16\}$ $\rightarrow$ the train batch size corresponds to the number of images the model trains on at each sub-iteration of a training epoch. 
- The learning rate ($lr$) $ \in [1e-5, 1e-3]$ $\rightarrow$ The learning rate or initial learning rate defines the amount of the gradients that will be used to update the weights. It is sent as an argument to the optimizer.
- The weight decay $ = 0$ $\rightarrow$ corresponds to a floating number that will be multiplied with the norm of the weights to diminish their effects or regularize them during the training. We didn't fix a weight since it doesn't provide us a great result at the first training but we kept the explanation on the weight decay usage.


The optimizer to use is the `AdamW` optimizer. It provides a smoother updating so that the weights do not diverge (stability of the training), and a weight decay is provided to regularize the training and not over-fit. `AdamW` was introduced in *Decoupled Weight Decay Regularization* (see [AdamW](https://arxiv.org/pdf/1711.05101)) by Ilya Loshchilov and Frank Hutter. It is an update to the original `Adam` optimizer, and the algorithm is defined as follows (pseudo-code with Python):

----------------
```python
def adamW(lr: float, betas: tuple (beta_1, beta_2), parameters: [matrix], loss: [function], epsilon: float, weight_decay: float, amsgrad: bool):

    """ Updating the parameters

    Args:
        lr (float): The learning rate
        betas (tuple): The step size of the momentums
        parameters ([matrix]): The weights or group of importance inside the model that we want to update
        loss ([function]): The loss function or objective
        epsilon (float): A number to add to the square root of the second momentum to avoid division with zero
        weight_decay (float): A float multiplied with the parameters to regularize it
        amsgrad (bool): Indicate if we want to normalize the first momentum using the maximum of the second momentum or not
    
    Returns:
        [matrix]: The updated parameters
    """
            
    # recuperate the betas
    beta_1, beta_2 = betas

    # initialize the first, second moments, and maximum second momentum
    m = v = v_max = 0

    # determine the gradient of the objective compared to the parameters or weights
    delta = grad(loss) / grad(parameters)

    # decay the weights by multiplying with the learning rate, and the weight decay
    parameters = parameters - wd*lr*parameters

    # update the first momentum
    m = beta_1*m + (1 - beta_1)*delta

    # update the second momentum
    v = beta_2*v + (1 - beta_2)*delta**2

    #  get the normalized first momentum
    m_ = m/(1 - beta_1)

    # get the normalized second momentum
    v_ = v/(1 - beta_2)

    if amsgrad: # if amsgrad is set to True
        
        # take the maximum between the normalized and the last maximum second momentum
        v_max = max(v_max, v_)

        # update the weights with the maximum second momentum
        parameters = parameters - lr*m_ / (sqrt(v_max) + epsilon)
    
    else: # if amsgrad is set to False

        # update the parameters with the normalized second momentum
        parameters = parameters - lr*m_ / (sqrt(v_) + epsilon)
    
    return parameters

```
------------------

Another thing to define is the learning rate scheduler. His role is to linearly augment the learning rate proportionally to several steps named the warmup steps and beginning to linearly decay after the warmup steps until reaching zero at the end of the training. A step increase at each time we update the parameters. That is the following pseudo-code (see the actual code [true_code](https://github.com/huggingface/transformers/blob/v4.29.1/src/transformers/optimization.py#L100)):

------------------
```python
def linear_learning_rate_scheduling_with_warmup(initial_lr: float, current_step: int, warmup_steps: int, max_training_step: int):
    """ Linearly decay the learning after a warmup steps

    Args:
        initial_lr (float): The initial learning rate of a specific group of weights
        current_step (int): The current step of the training
        warmup_steps (int): The number os steps before beginning to decay the learning rate
        max_training_step (int): The maximum number of training steps
    """

    if current_step < warmup_step: # if the warmup_steps is not yet reached 

        # We take a part of the initial learning proportional to the number of steps already reached before the warmup steps
        return initial_lr*float(current_step) / float(warmup_step)
    
    else: # if the warmup step is already reached

        # We take a part of the initial learning proportional to the number of steps that it remains to travel since the warmup steps
        return initial_lr*float(max_training_steps - current_step) / float(max_training_steps - warmup_step)

```
--------------


Let us initialize below the search spaces according to that defined in [bayesian_optimization](bayesian_optimization.ipynb):

In [2]:
search_spaces = {
    'lr': {
        'min': 1e-5,
        'max': 1e-4 # lower is better
    },
    'batch_size': {
        'value': 16
    },
    'h_flip_p': {
        'min': 0.0,
        'max': 0.5
    },
    'v_flip_p': {
        'min': 0.0,
        'max': 0.5
    },
    'gray_scale_p': {
        'min': 0.0,
        'max': 0.5
    },
    'rotation': {
        'values': [1, 0]
    }
}

In the next section, we will search for the best model using randomly generated samples from the search spaces.

### Search for the best model

In this section, we must load the ViT Model and the custom hugging face trainer we configured in [preprocessing_and_loading](preprocessing_and_loading.ipynb). Let us recuperate the two.

In [3]:
# recuperate the labels weights
with open('data/extractions/weights.txt', 'rb') as f:
    
    depick = pickle.Unpickler(f)
    
    weights = depick.load()
    
# recuperate the image characteristics
with open('data/extractions/fake_real_dict.txt', 'rb') as f:
    
    depick = pickle.Unpickler(f)
    
    characs = depick.load()

# define the model name
model_name = 'google/vit-base-patch16-224-in21k'

# initialize the model without calling it
model = partial(
    ViTForImageClassification.from_pretrained,
    model_name, 
    num_labels = len(characs['ids']),
    id2label = {name: key for key, name in characs['ids'].items()},
    label2id = characs['ids']
    )

# recuperate the trainer class
Trainer = get_custom_trainer(weights)

We must also define the training and validation datasets with the ViT Model's feature extractor as their Transformer. Only the training set requires an augmentation strategy. So we will make them inside a function.

In [4]:
# initialize the feature extractor
feature_extractor = ViTFeatureExtractor(model_name)

def get_datasets(h_flip_p: float, v_flip_p: float, gray_scale_p: float, rotation: bool):

    # define a empty function
    empty = lambda x: x
    
    # initialize the training transformer
    training_transformer = Compose([
        transforms.RandomHorizontalFlip(h_flip_p),
        transforms.RandomVerticalFlip(v_flip_p),
        transforms.RandomGrayscale(gray_scale_p),
        transforms.RandomRotation(degrees=(0, 90)) if rotation else empty,
        partial(feature_extractor, return_tensors = 'pt')
    ])
    
    # define the path
    path = 'data/real_and_fake_splits'
    
    # recuperate the training dataset
    train_dataset = FakeFaceDetectionDataset(f"{path}/train/training_fake",
                                             f"{path}/train/training_real",
                                             characs['ids'],
                                             training_transformer)
    
    # recuperate the validation dataset
    valid_dataset = FakeFaceDetectionDataset(f"{path}/valid/training_fake",
                                             f"{path}/valid/training_real",
                                             characs['ids'],
                                             feature_extractor, transformer_kwargs = {'return_tensors': 'pt'})
    
    return train_dataset, valid_dataset

Let us now create the training function below. We will use the `compute_metrics` function that we created in [vit_model](vit_model.ipynb) for the prediction to calculate the metrics and the `data_collator` that we created in [preprocessing_and_loading](preprocessing_and_loading.ipynb) to make the model load the images.

In [11]:
%%writefile fake-face-detection/fake_face_detection/trainers/search_train.py

from fake_face_detection.metrics.compute_metrics import compute_metrics
from fake_face_detection.data.collator import fake_face_collator
from transformers import Trainer, TrainingArguments, set_seed
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from typing import *
import numpy as np
import json
import os

def train(epochs: int, output_dir: str, config: dict, model: nn.Module, trainer, get_datasets: Callable, log_dir: str = "fake_face_logs", metric = 'accuracy', seed: int = 0):
    
    print("------------------------- Beginning of training")
    
    set_seed(seed)
    
    # initialize the model
    model = model()
    
    # reformat the config integer type
    for key, value in config.items():
        
        if isinstance(value, np.int32): config[key] = int(value)
    
    pretty = json.dumps(config, indent = 4)
    
    print(f"Current Config: \n {pretty}")
    
    print(f"Checkpoints in {output_dir}")
    
    # recuperate the dataset
    train_dataset, test_dataset = get_datasets(config['h_flip_p'], config['v_flip_p'], config['gray_scale_p'], config['rotation'])
    
    # initialize the arguments of the training
    training_args = TrainingArguments(output_dir,
                                      per_device_train_batch_size=config['batch_size'],
                                      evaluation_strategy='epoch',
                                      save_strategy='epoch',
                                      logging_strategy='epoch',
                                      num_train_epochs=epochs,
                                      fp16=True,
                                      save_total_limit=2,
                                      push_to_hub=False,
                                      logging_dir=os.path.join(log_dir, os.path.basename(output_dir)),
                                      load_best_model_at_end=True,
                                      learning_rate=config['lr']
                                      )
    
    # train the model
    trainer_ = trainer(
        model = model,
        args = training_args,
        data_collator = fake_face_collator,
        compute_metrics = compute_metrics,
        train_dataset = train_dataset,
        eval_dataset = test_dataset
    )
    
    # train the model
    trainer_.train()
    
    # evaluate the model and recuperate metrics
    metrics = trainer_.evaluate(test_dataset)
    
    # add metrics and config to the hyperparameter panel of tensorboard
    with SummaryWriter(os.path.join(log_dir, 'hparams')) as logger:
        
        logger.add_hparams(
            config, metrics
        )
    
    print(metrics)
    
    print("------------------------- End of training")
    # recuperate the metric to evaluate
    return metrics[f'eval_{metric}']
        

Overwriting fake-face-detection/fake_face_detection/trainers/search_train.py


In [6]:
%run fake-face-detection/fake_face_detection/trainers/search_train.py

We implemented a new Bayesian optimization function based on our original one ([bayesian_optimization](bayesian_optimization.ipynb)), and that can find the code here [bayesian_optimization_for_fake_face_pred](fake_face_bayesian_optimization.py). It will make many evaluation steps to find the combination of hyperparameters providing us with the best model. Notice that each trial will provide a model that must be locally saved. The number of epochs can be fixed to `4` and the number of attempts to `20`. Moreover, the scale size will be taken as default `0.1`. 

The `f1` will be chosen as our principal metric.

In [11]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}

# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Current Config: 
 {
    "lr": 9.689183977257256e-05,
    "batch_size": 16,
    "h_flip_p": 0.24296384828140632,
    "v_flip_p": 0.4591171658925659,
    "gray_scale_p": 0.4149264518294957,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_2yW4Acq


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
 25%|██▌       | 110/440 [01:53<05:13,  1.05it/s]

{'loss': 0.6821, 'learning_rate': 7.266887982942942e-05, 'epoch': 1.0}


...

{'eval_loss': 0.4269016683101654, 'eval_accuracy': 0.8054054054054054, 'eval_f1': 0.8105263157894738, 'eval_runtime': 10.0503, 'eval_samples_per_second': 18.407, 'eval_steps_per_second': 2.388, 'epoch': 4.0}


100%|██████████| 440/440 [10:04<00:00,  1.37s/it]


{'train_runtime': 605.1563, 'train_samples_per_second': 11.587, 'train_steps_per_second': 0.727, 'train_loss': 0.5650030482899059, 'epoch': 4.0}


100%|██████████| 24/24 [00:06<00:00,  3.80it/s]


{'eval_loss': 0.4269016683101654, 'eval_accuracy': 0.8054054054054054, 'eval_f1': 0.8105263157894738, 'eval_runtime': 6.9234, 'eval_samples_per_second': 26.721, 'eval_steps_per_second': 3.467, 'epoch': 4.0}
------------------------- End of training


Let us make 20 trials.

In [12]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_JulUpak'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 1.6528880304384158e-05,
    "batch_size": 16,
    "h_flip_p": 0.22914276130929306,
    "v_flip_p": 0.4992272204272212,
    "gray_scale_p": 0.4980482239275472,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_JulUpak


 25%|██▌       | 110/440 [02:06<05:37,  1.02s/it]

{'loss': 0.6833, 'learning_rate': 1.2396660228288119e-05, 'epoch': 1.0}


...

{'eval_loss': 0.4256192445755005, 'eval_accuracy': 0.7945945945945946, 'eval_f1': 0.806122448979592, 'eval_runtime': 8.5378, 'eval_samples_per_second': 21.668, 'eval_steps_per_second': 2.811, 'epoch': 4.0}


100%|██████████| 440/440 [09:20<00:00,  1.27s/it]


{'train_runtime': 560.6743, 'train_samples_per_second': 12.506, 'train_steps_per_second': 0.785, 'train_loss': 0.5145724036476829, 'epoch': 4.0}


100%|██████████| 24/24 [00:06<00:00,  3.51it/s]


{'eval_loss': 0.4256192445755005, 'eval_accuracy': 0.7945945945945946, 'eval_f1': 0.806122448979592, 'eval_runtime': 7.2337, 'eval_samples_per_second': 25.575, 'eval_steps_per_second': 3.318, 'epoch': 4.0}
------------------------- End of training


-------------

In [13]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 2


In [14]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_Md7dObb'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 8.066164211147095e-05,
    "batch_size": 16,
    "h_flip_p": 0.14262461447346014,
    "v_flip_p": 0.34829575180008127,
    "gray_scale_p": 0.365252658707733,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_Md7dObb


 25%|██▌       | 110/440 [02:06<05:37,  1.02s/it]

{'loss': 0.6865, 'learning_rate': 6.049623158360321e-05, 'epoch': 1.0}


...

{'eval_loss': 0.43839508295059204, 'eval_accuracy': 0.7837837837837838, 'eval_f1': 0.8058252427184466, 'eval_runtime': 6.7672, 'eval_samples_per_second': 27.338, 'eval_steps_per_second': 3.547, 'epoch': 4.0}


100%|██████████| 440/440 [08:31<00:00,  1.16s/it]


{'train_runtime': 511.1208, 'train_samples_per_second': 13.719, 'train_steps_per_second': 0.861, 'train_loss': 0.5372205560857599, 'epoch': 4.0}


100%|██████████| 24/24 [00:06<00:00,  3.72it/s]

{'eval_loss': 0.43839508295059204, 'eval_accuracy': 0.7837837837837838, 'eval_f1': 0.8058252427184466, 'eval_runtime': 6.9224, 'eval_samples_per_second': 26.725, 'eval_steps_per_second': 3.467, 'epoch': 4.0}
------------------------- End of training


-------------

In [15]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 4


In [16]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_tD65Tok'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 8.066164211147095e-05,
    "batch_size": 16,
    "h_flip_p": 0.14262461447346014,
    "v_flip_p": 0.34829575180008127,
    "gray_scale_p": 0.365252658707733,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_tD65Tok


 25%|██▌       | 110/440 [02:15<05:38,  1.03s/it]

{'loss': 0.6867, 'learning_rate': 6.067955349749292e-05, 'epoch': 1.0}


...

{'eval_loss': 0.39084506034851074, 'eval_accuracy': 0.8216216216216217, 'eval_f1': 0.8324873096446701, 'eval_runtime': 7.7062, 'eval_samples_per_second': 24.007, 'eval_steps_per_second': 3.114, 'epoch': 4.0}


100%|██████████| 440/440 [08:11<00:00,  1.12s/it]


{'train_runtime': 491.5286, 'train_samples_per_second': 14.266, 'train_steps_per_second': 0.895, 'train_loss': 0.5208114363930442, 'epoch': 4.0}


100%|██████████| 24/24 [00:06<00:00,  3.80it/s]


{'eval_loss': 0.39084506034851074, 'eval_accuracy': 0.8216216216216217, 'eval_f1': 0.8324873096446701, 'eval_runtime': 6.592, 'eval_samples_per_second': 28.064, 'eval_steps_per_second': 3.641, 'epoch': 4.0}
------------------------- End of training


-------------

In [17]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 6


In [18]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_HZUsA7R'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 6.459345257343661e-05,
    "batch_size": 16,
    "h_flip_p": 0.4821814530037276,
    "v_flip_p": 0.4644567465010884,
    "gray_scale_p": 0.37763259831790275,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_HZUsA7R


 25%|██▌       | 110/440 [01:57<05:19,  1.03it/s]

{'loss': 0.6627, 'learning_rate': 4.844508943007745e-05, 'epoch': 1.0}


...

{'eval_loss': 0.3955724835395813, 'eval_accuracy': 0.827027027027027, 'eval_f1': 0.8367346938775511, 'eval_runtime': 6.3472, 'eval_samples_per_second': 29.147, 'eval_steps_per_second': 3.781, 'epoch': 4.0}


100%|██████████| 440/440 [06:52<00:00,  1.07it/s]


{'train_runtime': 412.9509, 'train_samples_per_second': 16.98, 'train_steps_per_second': 1.066, 'train_loss': 0.5161731719970704, 'epoch': 4.0}


100%|██████████| 24/24 [00:05<00:00,  4.06it/s]

{'eval_loss': 0.3955724835395813, 'eval_accuracy': 0.827027027027027, 'eval_f1': 0.8367346938775511, 'eval_runtime': 6.1506, 'eval_samples_per_second': 30.078, 'eval_steps_per_second': 3.902, 'epoch': 4.0}
------------------------- End of training


-------------

In [19]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 8


In [20]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_qnt7qTm'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 6.418168561449824e-05,
    "batch_size": 16,
    "h_flip_p": 0.2806225314693065,
    "v_flip_p": 0.35800980646120173,
    "gray_scale_p": 0.35066248679511797,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_qnt7qTm


 25%|██▌       | 110/440 [01:21<03:35,  1.53it/s]

{'loss': 0.6597, 'learning_rate': 4.813626421087368e-05, 'epoch': 1.0}


...

{'eval_loss': 0.47780826687812805, 'eval_accuracy': 0.7513513513513513, 'eval_f1': 0.7578947368421053, 'eval_runtime': 6.2008, 'eval_samples_per_second': 29.835, 'eval_steps_per_second': 3.87, 'epoch': 4.0}


100%|██████████| 440/440 [06:40<00:00,  1.10it/s]


{'train_runtime': 400.5351, 'train_samples_per_second': 17.507, 'train_steps_per_second': 1.099, 'train_loss': 0.5750896280462091, 'epoch': 4.0}


100%|██████████| 24/24 [00:05<00:00,  4.15it/s]

{'eval_loss': 0.47780826687812805, 'eval_accuracy': 0.7513513513513513, 'eval_f1': 0.7578947368421053, 'eval_runtime': 6.1407, 'eval_samples_per_second': 30.127, 'eval_steps_per_second': 3.908, 'epoch': 4.0}
------------------------- End of training


-------------

In [21]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 10


In [22]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_XsNJh3e'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 5.08069272131592e-05,
    "batch_size": 16,
    "h_flip_p": 0.41705521332037515,
    "v_flip_p": 0.0813270485780424,
    "gray_scale_p": 0.17763535011376075,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_XsNJh3e


 25%|██▌       | 110/440 [01:36<04:43,  1.16it/s]

{'loss': 0.6637, 'learning_rate': 3.81051954098694e-05, 'epoch': 1.0}


...

{'eval_loss': 0.5319870114326477, 'eval_accuracy': 0.745945945945946, 'eval_f1': 0.7539267015706805, 'eval_runtime': 6.3274, 'eval_samples_per_second': 29.238, 'eval_steps_per_second': 3.793, 'epoch': 4.0}


100%|██████████| 440/440 [06:41<00:00,  1.09it/s]


{'train_runtime': 402.1036, 'train_samples_per_second': 17.438, 'train_steps_per_second': 1.094, 'train_loss': 0.5950130289251154, 'epoch': 4.0}


100%|██████████| 24/24 [00:06<00:00,  3.90it/s]

{'eval_loss': 0.5319870114326477, 'eval_accuracy': 0.745945945945946, 'eval_f1': 0.7539267015706805, 'eval_runtime': 6.5434, 'eval_samples_per_second': 28.273, 'eval_steps_per_second': 3.668, 'epoch': 4.0}
------------------------- End of training


-------------

In [23]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 12


In [24]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_KxxdKWh'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 1.6166919943341313e-05,
    "batch_size": 16,
    "h_flip_p": 0.03398043168787862,
    "v_flip_p": 0.4309096009850647,
    "gray_scale_p": 0.20188777565913874,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_KxxdKWh


 25%|██▌       | 110/440 [01:22<03:35,  1.53it/s]

{'loss': 0.6814, 'learning_rate': 1.2125189957505985e-05, 'epoch': 1.0}


...

{'eval_loss': 0.4600895941257477, 'eval_accuracy': 0.7891891891891892, 'eval_f1': 0.8, 'eval_runtime': 6.2404, 'eval_samples_per_second': 29.646, 'eval_steps_per_second': 3.846, 'epoch': 4.0}


100%|██████████| 440/440 [07:08<00:00,  1.03it/s]


{'train_runtime': 428.7112, 'train_samples_per_second': 16.356, 'train_steps_per_second': 1.026, 'train_loss': 0.5417569333856757, 'epoch': 4.0}


100%|██████████| 24/24 [00:05<00:00,  4.08it/s]

{'eval_loss': 0.4600895941257477, 'eval_accuracy': 0.7891891891891892, 'eval_f1': 0.8, 'eval_runtime': 6.1238, 'eval_samples_per_second': 30.21, 'eval_steps_per_second': 3.919, 'epoch': 4.0}
------------------------- End of training


-------------

In [25]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 14


In [26]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_Tr1WYDq'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 7.829266304167906e-05,
    "batch_size": 16,
    "h_flip_p": 0.07982965818844506,
    "v_flip_p": 0.21130719907675127,
    "gray_scale_p": 0.13893567083582092,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_Tr1WYDq


 25%|██▌       | 110/440 [01:27<04:14,  1.30it/s]

{'loss': 0.6535, 'learning_rate': 5.8719497281259293e-05, 'epoch': 1.0}


...

{'eval_loss': 0.35217583179473877, 'eval_accuracy': 0.8594594594594595, 'eval_f1': 0.8659793814432989, 'eval_runtime': 8.9829, 'eval_samples_per_second': 20.595, 'eval_steps_per_second': 2.672, 'epoch': 4.0}


100%|██████████| 440/440 [08:50<00:00,  1.20s/it]


{'train_runtime': 530.346, 'train_samples_per_second': 13.222, 'train_steps_per_second': 0.83, 'train_loss': 0.45588083267211915, 'epoch': 4.0}


100%|██████████| 24/24 [00:07<00:00,  3.16it/s]

{'eval_loss': 0.35217583179473877, 'eval_accuracy': 0.8594594594594595, 'eval_f1': 0.8659793814432989, 'eval_runtime': 8.432, 'eval_samples_per_second': 21.94, 'eval_steps_per_second': 2.846, 'epoch': 4.0}
------------------------- End of training


-------------

In [27]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 16


In [28]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_5yTGwaH'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 1.064717550865842e-05,
    "batch_size": 16,
    "h_flip_p": 0.35392047327422704,
    "v_flip_p": 0.0294368834381617,
    "gray_scale_p": 0.03370016405515375,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_5yTGwaH


 25%|██▌       | 110/440 [01:52<05:32,  1.01s/it]

{'loss': 0.6827, 'learning_rate': 7.985381631493814e-06, 'epoch': 1.0}


...

{'eval_loss': 0.42529231309890747, 'eval_accuracy': 0.8216216216216217, 'eval_f1': 0.835820895522388, 'eval_runtime': 7.9565, 'eval_samples_per_second': 23.252, 'eval_steps_per_second': 3.016, 'epoch': 4.0}


100%|██████████| 440/440 [10:02<00:00,  1.37s/it]


{'train_runtime': 602.74, 'train_samples_per_second': 11.634, 'train_steps_per_second': 0.73, 'train_loss': 0.4930061947215687, 'epoch': 4.0}


100%|██████████| 24/24 [00:08<00:00,  2.87it/s]

{'eval_loss': 0.42529231309890747, 'eval_accuracy': 0.8216216216216217, 'eval_f1': 0.835820895522388, 'eval_runtime': 8.8189, 'eval_samples_per_second': 20.978, 'eval_steps_per_second': 2.721, 'epoch': 4.0}
------------------------- End of training


-------------

In [7]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 18


In [8]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_OZe30nD'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Current Config: 
 {
    "lr": 9.474979774366729e-05,
    "batch_size": 16,
    "h_flip_p": 0.14995067958799224,
    "v_flip_p": 0.2712280698131356,
    "gray_scale_p": 0.09845885146155786,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_OZe30nD


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
 25%|██▌       | 110/440 [02:00<05:14,  1.05it/s]

{'loss': 0.6807, 'learning_rate': 7.106234830775047e-05, 'epoch': 1.0}


...

{'eval_loss': 0.32448598742485046, 'eval_accuracy': 0.8594594594594595, 'eval_f1': 0.8673469387755102, 'eval_runtime': 7.1509, 'eval_samples_per_second': 25.871, 'eval_steps_per_second': 3.356, 'epoch': 4.0}


100%|██████████| 440/440 [09:52<00:00,  1.35s/it]


{'train_runtime': 593.6357, 'train_samples_per_second': 11.812, 'train_steps_per_second': 0.741, 'train_loss': 0.47146494605324485, 'epoch': 4.0}


100%|██████████| 24/24 [00:07<00:00,  3.20it/s]


{'eval_loss': 0.32448598742485046, 'eval_accuracy': 0.8594594594594595, 'eval_f1': 0.8673469387755102, 'eval_runtime': 8.0536, 'eval_samples_per_second': 22.971, 'eval_steps_per_second': 2.98, 'epoch': 4.0}
------------------------- End of training


-------------

In [9]:
EPOCHS = 4
TRIALS = 2 # We will make the trial 2 by 2 until reaching the desired number of trials 

# let us initialize the arguments before the training function be fed 
kwargs = {'epochs': EPOCHS, 
        'model': model, 
        'trainer': Trainer,
        'get_datasets': get_datasets,
        'seed': 0,
        'metric': 'f1'}


# let us instantiate the search function we will indicate the output dir as random kwarg initialization of the trainer
bo_search = SimpleBayesianOptimizationForFakeReal(train, search_spaces, random_kwargs = {'output_dir': 'data/checkpoints/model_'}, kwargs = kwargs, checkpoint='data/trials/checkpoint_2.txt')

Checkpoint loaded at trial 20


In [10]:
bo_search.optimize(n_trials=TRIALS)

{'output_dir': 'data/checkpoints/model_Xll9TtZ'}
------------------------- Beginning of training


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` envir

Current Config: 
 {
    "lr": 1.2827166224771633e-05,
    "batch_size": 16,
    "h_flip_p": 0.16521424454024936,
    "v_flip_p": 0.25707805990429455,
    "gray_scale_p": 0.1392386319822735,
    "rotation": 1
}
Checkpoints in data/checkpoints/model_Xll9TtZ


 25%|██▌       | 110/440 [01:54<05:59,  1.09s/it]

{'loss': 0.6806, 'learning_rate': 9.620374668578724e-06, 'epoch': 1.0}


...

{'eval_loss': 0.3556261360645294, 'eval_accuracy': 0.8594594594594595, 'eval_f1': 0.8645833333333334, 'eval_runtime': 6.6377, 'eval_samples_per_second': 27.871, 'eval_steps_per_second': 3.616, 'epoch': 4.0}


100%|██████████| 440/440 [09:56<00:00,  1.36s/it]


{'train_runtime': 596.6632, 'train_samples_per_second': 11.752, 'train_steps_per_second': 0.737, 'train_loss': 0.4487828991629861, 'epoch': 4.0}


100%|██████████| 24/24 [00:06<00:00,  3.77it/s]

{'eval_loss': 0.3556261360645294, 'eval_accuracy': 0.8594594594594595, 'eval_f1': 0.8645833333333334, 'eval_runtime': 6.5768, 'eval_samples_per_second': 28.129, 'eval_steps_per_second': 3.649, 'epoch': 4.0}
------------------------- End of training


The best model is located at `data/checkpoints/model_lhGqMDq` with an accuracy of <i style="color: orange">85.946</i> and an f1_score of <i style="color: orange">86.735</i>.

Let us make some predictions on the set in [predictions](predictions.ipynb)